In [327]:
import pandas as pd 
import numpy as np
import csv,re
import os,io,sys





In [328]:
d_drug_french={'AMK':'Amikacin' ,  'BDQ':'Bédaquiline','CAP':'Capréomycin','CFZ':'Clofazimine', 'DLM':'Délamanid','Pa':'Prétomanide', 'EMB':'Ethambutol', 'ETO':'Ethionamide', 'INH':'Isoniazid','KAN':'Kanamycin','LFX':'Levofloxacin','LNZ':'Linézolid','MFX':'Moxifloxacin','PZA':'Pyrazinamide','RIF':'Rifampicin' , 'STR':'Streptomycin', 'FQ':'Fluoroquinolones', 'CIP':'Ciprofloxacin', 'OFX':'Ofloxacin' , 'Para-aminosalicylic':'PAS', 'AMG':'Aminoglycosides',  'CYS':'Cyclosérine', 'Para-aminosalicylic_acid' :'PAS', 'PAS':'PAS' }

d_drug_english={'AMK':'Amikacin' ,  'BDQ':'Bedaquiline','CAP':'Capreomycin','CFZ':'Clofazimine', 'DLM':'Delamanid','Pa':'Pretomanide', 'EMB':'Ethambutol', 'ETO':'Ethionamide', 'INH':'Isoniazid','KAN':'Kanamycin','LFX':'Levofloxacin','LNZ':'Linezolid','MFX':'Moxifloxacin','PZA':'Pyrazinamide','RIF':'Rifampicin' , 'STR':'Streptomycin', 'FQ':'Fluoroquinolones', 'CIP':'Ciprofloxacin', 'OFX':'Ofloxacin', 'AMG':'Aminoglycosides',  'CYS':'Cycloserine', 'PAS':'PAS', 'CYC':'Cycloserine' }
d_drug_french['BDQ']


'Bédaquiline'

In [329]:
d_drug_english['BDQ']

'Bedaquiline'

In [330]:

#df1= pd.read_csv("/home/azadeh/workspace/snakemake_tutorial/snakemake-snakemake-tutorial-data-f17b467/output_Run_57/2412067826KIA_lib/2412067826KIA_lib_join_table_dailyPipeline_Tbprofiler_MTBseq_NOTFiltered.csv", delimiter="\t") #pipeline
#/home/azadeh/workspace/snakemake_tutorial/snakemake-snakemake-tutorial-data-f17b467/script_reformating_graphicInterface_after_merge/2408056426BEL_S48_join_table_dailyPipeline_Tbprofiler_MTBseq_NOTFiltered.csv
df1=pd.read_csv("/home/azadeh/workspace/snakemake_tutorial/snakemake-snakemake-tutorial-data-f17b467/output_Run_59_bizz/2412075340OCH_lib/2412075340OCH_lib_join_table_dailyPipeline_Tbprofiler_MTBseq_NOTFiltered.csv", delimiter="\t")


#place_to_write=os.path.dirname(sys.argv[1])+"/"

#df1=pd.read_csv(sys.argv[1], delimiter="\t")
#path_table1_toWrite=sys.argv[2]
#path_table1_toWriteExcel=sys.argv[2].replace(".csv",".xlsx")
#path_table2_toWrite=sys.argv[3]
#path_table3_toWrite=sys.argv[4]



#df1=pd.read_csv(sys.argv[1], delimiter="\t")

df1=df1.drop(columns=['N_WHOALL_R', 'N_WHOALL_S','GENEID', 'REF', 'ALT'])
#df1=df1.drop(columns=['GENEID', 'REF', 'ALT'])


df1['ANTIBIO']=df1['ANTIBIO'].apply(lambda x: d_drug_english[x] if  pd.notnull(x) else "Na")
def mut_synonym(x):
    return(x[0]==x[-1] and x!="NaN" and x[0] in ['C','D','S','Q','K','I', 'P', 'T','F', 'N','G', 'H', 'L', 'R', 'W', 'A','V','E','Y','M'])
 
x='L243L'
x[0]
x[-1]
mut_synonym(x)


True

In [331]:

dfreformat=df1.rename(columns={'ANTIBIO':'Antibiotique' , 'GENE':'Gène' , 'POS':'Position génomique' , 'HGVS_C':'Mutation (nucléotide)', 'HGVS_P':'Mutation (AA 1 lettre)', 'HGVS_P_3L':'Mutation (AA 3 lettres)','Freq_Mutated':'Fréquence','DP': 'Profondeur', 'P_INTER':'PhyResSE', 'W_INTER':'Walker', 'Res_TBprof':'Tbprofiler','Res_MTBseq':'MTBseq','WHOALL_GRADING':'OMS'})
dfreformat
dfreformat=dfreformat.loc[(dfreformat["Gène"]!="ddlA" ) | (dfreformat["Mutation (AA 1 lettre)"]!="T365A") ]
dfreformat
dfreformat['Mutation (AA 1 lettre)']=dfreformat.apply( lambda x: "True" if (mut_synonym(str(x[5])) and (str(x[4])=="fabG1" or str(x[5])=="L203L")) else x[5] , axis=1)

dfreformat['Mutation (AA 1 lettre)']
dfreformat=dfreformat.loc[dfreformat['Mutation (AA 1 lettre)']!="True"]
#
#dfreformat['Mutation (AA 1 lettre)']
#dfreformat


In [332]:
dfreformat_table_2=dfreformat[['Antibiotique','Gène', 'Position génomique','Mutation (nucléotide)','Mutation (AA 1 lettre)', 'Mutation (AA 3 lettres)', 'Fréquence', 'Profondeur', 'PhyResSE', 'Walker' , 'Tbprofiler','MTBseq', 'OMS' ]]

dfreformat_table_2['MTBseq']=dfreformat_table_2['MTBseq'].replace("ND" , "Non détecté")
dfreformat_table_2['MTBseq']=dfreformat_table_2['MTBseq'].replace("Res" , "Résistant")
dfreformat_table_2['MTBseq']=dfreformat_table_2['MTBseq'].apply(lambda x : "Phylogénie" if re.match(r"phylo \(.*\)", str(x)) or x=="Polymorphism" else x)

dfreformat_table_2['Tbprofiler']=dfreformat_table_2['Tbprofiler'].replace("ND" , "Non détecté")
dfreformat_table_2['Tbprofiler']=dfreformat_table_2['Tbprofiler'].replace("Res" , "Résistant")

dfreformat_table_2['Walker']=dfreformat_table_2['Walker'].replace("Resistant" , "Résistant")
dfreformat_table_2['Walker']=dfreformat_table_2['Walker'].replace("Uncharacterised" , "Non détecté")
dfreformat_table_2['Walker']=dfreformat_table_2['Walker'].apply(lambda x : "Phylogénie" if x in ["Benign", "phylo" ] else x)

dfreformat_table_2['PhyResSE']=dfreformat_table_2['PhyResSE'].replace("Resistant" , "Résistant").replace("CONF","Résistant")
dfreformat_table_2['PhyResSE']=dfreformat_table_2['PhyResSE'].apply(lambda x : "Phylogénie" if re.match(r"phylo \(.*\)", str(x)) or x=="Polymorphism" else x)
dfreformat_table_2['Mutation (nucléotide)']=dfreformat_table_2['Mutation (nucléotide)'].apply(lambda x : str(x).lower())

#dfreformat_table_2.to_csv(path_table3_toWrite, sep='\t', index=False)

dfreformat_table_2
#dfreformat=dfreformat1


,Antibiotique,Gène,Position génomique,Mutation (nucléotide),Mutation (AA 1 lettre),Mutation (AA 3 lettres),Fréquence,Profondeur,PhyResSE,Walker,Tbprofiler,MTBseq,OMS
0,Amikacin,ccsA,620625,a735g,I245M,ILE245MET,100%,101.0,NaN,NaN,,Non détecté,5)_Not_assoc_w_R
1,Amikacin,rrs,1472070,nan,225,225,11.7647%,17.0,NaN,NaN,,,NaN
2,Amikacin,rrs,1472158,nan,313,313,13.3333%,15.0,NaN,NaN,,,NaN
3,Amikacin,rrs,1472874,nan,1029,1029,22.2222%,9.0,NaN,NaN,,,NaN
4,Amikacin,eis,2715342,g-10c,NaN,NaN,100%,121.0,NaN,NaN,Non détecté,,3)_Uncertain_significance
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Streptomycin,rrs,1472070,nan,225,225,11.7647%,17.0,NaN,NaN,,,NaN
96,Streptomycin,rrs,1472158,nan,313,313,13.3333%,15.0,NaN,NaN,,,NaN
97,Streptomycin,rrs,1472874,nan,1029,1029,22.2222%,9.0,NaN,NaN,,,NaN
98,Streptomycin,gid,4407588,a615g,A205A,ALA205ALA,100%,132.0,"phylo_(M.africanum,M.bovis,M.canettii,M.caprae...",NaN,Non détecté,Phylogénie,5)_Not_assoc_w_R


In [333]:

def tbprof_res(x) :
    tbprof=x[10]
       
    if tbprof=="Res":
        return ("Résistant" , "tbprofiler")
    else:
        return ("INCONNU", "tbprofiler")

def mtbseq_res(x) :
    mtbseq=x[11]    
    if mtbseq=="Res":
        return ("Résistant" , "mtbseq")
    if re.match(r"phylo \(.*\)", str(mtbseq)) or mtbseq=="Polymorphism" :
        return ("Sensible" , "mtbseq")
    else: #ND or NAN 
        return ("INCONNU" , "mtbseq")
        

def walker_res(x) :
    walker=x[9]    
    if walker=="Resistant":
        return ("Résistant" , "walker")
    if walker in ["Benign", "phylo" ] :
        return ("Sensible" , "walker")
    else:
        return ("INCONNU" , "walker")

def phy_res(x) :
    phyr=str(x[8])
    if phyr in ["Resistant" , "CONF"]:
        return ("Résistant" , "PhyResSE")
    if re.match(r"phylo_\(.*\)", str(phyr)) or phyr=="Polymorphism" :
        return ("Sensible" , "PhyResSE")
    else:
        return ("INCONNU" , "PhyResSE")


#a=test4
#tbprof_res(a)
#mtbseq_res(a)
#walker_res(a)
#phy_res("Resistant")



In [334]:
def interpretation_source(x):
    
    oms=x[12]
    #print(oms)
    oms
    match oms: 
        case "1)_Assoc_w_R" | "2)_Assoc_w_R_-_Interim" : 
            return ("Résistant" , "OMS")
            
        case "4)_Not_assoc_w_R_-_Interim" | "5)_Not_assoc_w_R" : 
            return ("Sensible" , "OMS") 
          
        case _:
            tbprofres=tbprof_res(x)
            mtbseqres= mtbseq_res(x)
            phyres=phy_res(x)
            walkerres=walker_res(x)

            if  "Résistant" in tbprofres:
                return tbprofres
            if  "Résistant" in mtbseqres:
                return mtbseqres
            if  "Résistant" in phyres:
                return phyres 
            if  "Résistant" in walkerres:
                return walkerres
            else:
                if "Sensible" in mtbseqres:
                    return mtbseqres
                if "Sensible" in phyres:
                    return phyres
                if  "Sensible" in walkerres:
                    return walkerres
                else:
                    if oms =="3)_Uncertain_significance" :
                        return ("Indeterminé" , "all") 
                    else:
                        return ("INCONNU" , "all")
                                   
                


In [335]:
#test=dfreformat[dfreformat.Antibiotique=="Cycloserine"]
#test.Gène
#nex_test=test[test.Gène=="ddlA"]
#phy_res(nex_test.iloc[0])
#interpretation_source(nex_test.iloc[0])


In [336]:
#test['Interprétation']=test.apply(lambda x: interpretation_source(x)[0],axis=1)
#test['Source']=test.apply(lambda x: interpretation_source(x)[1],axis=1)
#test_table2=test.drop(columns=['Mutation (AA 3 lettres)','Profondeur', 'PhyResSE',	'Walker',	'Tbprofiler',	'MTBseq', 'OMS' ])
#test_table2
dfreformat

,Antibiotique,Position génomique,Profondeur,Fréquence,Gène,Mutation (AA 1 lettre),Mutation (AA 3 lettres),Mutation (nucléotide),PhyResSE,Walker,Tbprofiler,MTBseq,OMS
0,Amikacin,620625,101.0,100%,ccsA,I245M,ILE245MET,A735G,NaN,NaN,,ND,5)_Not_assoc_w_R
1,Amikacin,1472070,17.0,11.7647%,rrs,225,225,NaN,NaN,NaN,,,NaN
2,Amikacin,1472158,15.0,13.3333%,rrs,313,313,NaN,NaN,NaN,,,NaN
3,Amikacin,1472874,9.0,22.2222%,rrs,1029,1029,NaN,NaN,NaN,,,NaN
4,Amikacin,2715342,121.0,100%,eis,NaN,NaN,G-10C,NaN,NaN,ND,,3)_Uncertain_significance
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Streptomycin,1472070,17.0,11.7647%,rrs,225,225,NaN,NaN,NaN,,,NaN
96,Streptomycin,1472158,15.0,13.3333%,rrs,313,313,NaN,NaN,NaN,,,NaN
97,Streptomycin,1472874,9.0,22.2222%,rrs,1029,1029,NaN,NaN,NaN,,,NaN
98,Streptomycin,4407588,132.0,100%,gid,A205A,ALA205ALA,A615G,"phylo_(M.africanum,M.bovis,M.canettii,M.caprae...",NaN,ND,"phylo (M. africanum, M. bovis, M. canettii, M....",5)_Not_assoc_w_R


In [337]:
dfreformat['Mutation (nucléotide)']=dfreformat['Mutation (nucléotide)'].apply(lambda x : str(x).lower())
dfreformat
dfreformat['Interprétation']=dfreformat.apply(lambda x: interpretation_source(x)[0],axis=1)
dfreformat['Source']=dfreformat.apply(lambda x: interpretation_source(x)[1],axis=1)
dfreformat_table2=dfreformat.drop(columns=['Mutation (AA 3 lettres)','Profondeur', 'PhyResSE',	'Walker',	'Tbprofiler',	'MTBseq', 'OMS' ])

dfreformat_table2

,Antibiotique,Position génomique,Fréquence,Gène,Mutation (AA 1 lettre),Mutation (nucléotide),Interprétation,Source
0,Amikacin,620625,100%,ccsA,I245M,a735g,Sensible,OMS
1,Amikacin,1472070,11.7647%,rrs,225,nan,INCONNU,all
2,Amikacin,1472158,13.3333%,rrs,313,nan,INCONNU,all
3,Amikacin,1472874,22.2222%,rrs,1029,nan,INCONNU,all
4,Amikacin,2715342,100%,eis,NaN,g-10c,Indeterminé,all
...,...,...,...,...,...,...,...,...
95,Streptomycin,1472070,11.7647%,rrs,225,nan,INCONNU,all
96,Streptomycin,1472158,13.3333%,rrs,313,nan,INCONNU,all
97,Streptomycin,1472874,22.2222%,rrs,1029,nan,INCONNU,all
98,Streptomycin,4407588,100%,gid,A205A,a615g,Sensible,OMS


In [338]:
#dfreformat_table2.to_csv(path_table2_toWrite, sep='\t', index=False) #tableau 2

In [339]:

lis_of_gene_for_print=['eis', 'rrs', 'atpE' , 'mmpL5' , 'mmpS5' , 'pepQ', 'Rv0678', 'tlyA' , 'ddn', 'fbiA' , 'fbiB' ,'fbiC','fgd1' ,'fbiD', 'Rv2983','embB' , 'ethA', 'inhA','ahpC', 'dnaA', 'glpK', 'hadA', 'katG', 'mshA', 'ndh', 'Rv0010c', 'Rv1129c', 'Rv1258c', 'Rv2752c', 'gyrA','gyrB', 'rplC' , 'rrl', 'clpC1' , 'panD', 'pncA', 'PPE35', 'rpsA', 'Rv3236c', 'sigE', 'rpoB', 'gid', 'rpsL']

table1_step1=dfreformat_table2.drop(columns=['Position génomique', 'Fréquence',	'Source'])

table1_step1_sansNa=table1_step1[~ table1_step1.Antibiotique.isin(['Fluoroquinolones' , 'Na']) ]
table1_step1_sansNa=table1_step1 [table1_step1.Gène.isin(lis_of_gene_for_print)  ]

table1_step1_sansNa
#ordonner le tableau



,Antibiotique,Gène,Mutation (AA 1 lettre),Mutation (nucléotide),Interprétation
1,Amikacin,rrs,225,nan,INCONNU
2,Amikacin,rrs,313,nan,INCONNU
3,Amikacin,rrs,1029,nan,INCONNU
4,Amikacin,eis,NaN,g-10c,Indeterminé
7,Bedaquiline,mmpL5,I948V,a2842g,Sensible
...,...,...,...,...,...
95,Streptomycin,rrs,225,nan,INCONNU
96,Streptomycin,rrs,313,nan,INCONNU
97,Streptomycin,rrs,1029,nan,INCONNU
98,Streptomycin,gid,A205A,a615g,Sensible


In [340]:
def present_mutation(x):
    if pd.notna(x[2]) :
        return str(x[1])+" "+str(x[2])
    else:
        return str(x[1])+" "+str(x[3])

table1_step1_sansNa['Mutation']=table1_step1_sansNa.apply(lambda x: present_mutation(x) if str(x[4])!="Sensible" else "" , axis=1)
table1_step1_sansNa
#table1_NoSensible=table1_afterNA[table1_afterNA['Interprétation']!="SENSIBLE"]
#table1_NoSensible


/tmp/ipykernel_237560/3182372331.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table1_step1_sansNa['Mutation']=table1_step1_sansNa.apply(lambda x: present_mutation(x) if str(x[4])!="Sensible" else "" , axis=1)


,Antibiotique,Gène,Mutation (AA 1 lettre),Mutation (nucléotide),Interprétation,Mutation
1,Amikacin,rrs,225,nan,INCONNU,rrs 225
2,Amikacin,rrs,313,nan,INCONNU,rrs 313
3,Amikacin,rrs,1029,nan,INCONNU,rrs 1029
4,Amikacin,eis,NaN,g-10c,Indeterminé,eis g-10c
7,Bedaquiline,mmpL5,I948V,a2842g,Sensible,
...,...,...,...,...,...,...
95,Streptomycin,rrs,225,nan,INCONNU,rrs 225
96,Streptomycin,rrs,313,nan,INCONNU,rrs 313
97,Streptomycin,rrs,1029,nan,INCONNU,rrs 1029
98,Streptomycin,gid,A205A,a615g,Sensible,


In [341]:
table_for_interpretation_temp=table1_step1_sansNa.drop(columns=['Gène','Mutation (AA 1 lettre)', 'Mutation (nucléotide)'])
table_for_interpretation_temp.reset_index(drop=True, inplace=True)
table_for_interpretation_temp
#table_for_interpretation_temp.Antibiotique


,Antibiotique,Interprétation,Mutation
0,Amikacin,INCONNU,rrs 225
1,Amikacin,INCONNU,rrs 313
2,Amikacin,INCONNU,rrs 1029
3,Amikacin,Indeterminé,eis g-10c
4,Bedaquiline,Sensible,
...,...,...,...
62,Streptomycin,INCONNU,rrs 225
63,Streptomycin,INCONNU,rrs 313
64,Streptomycin,INCONNU,rrs 1029
65,Streptomycin,Sensible,


In [342]:
antibio_diag_sorted={'Rifampicin':'Sensible','Isoniazid':'Sensible','Pyrazinamide':'Sensible','Ethambutol':'Sensible','Streptomycin':'Sensible','Moxifloxacin':'Sensible','Levofloxacin':'Sensible','Bedaquiline':'Sensible','Clofazimine':'Sensible','Linezolid':'Sensible','Cycloserine' :'Sensible','Delamanid':'Sensible','Pretomanide':'Sensible', 'Kanamycin': 'Sensible', 'Amikacin':'Sensible', 'Capreomycin':'Sensible', 'Ethionamide':'Sensible','PAS':'Sensible'}
                                                                                               
order=list(antibio_diag_sorted.keys())
antibio_of_x_list=table_for_interpretation_temp.Antibiotique.to_list()

def adding_sensible_antibiotiques(x):

    
    for element in order:
        print(element)
        if element not in antibio_of_x_list:
            ind=order.index(element)
            print(element)
            #print(ind)
            new_row=[element, "Sensible",'']
            print(len(x))
            x.loc[len(x)]=new_row
            print(x)



#table_for_interpretation_temp.Antibiotique


adding_sensible_antibiotiques(table_for_interpretation_temp)

#table_for_interpretation_temp

Rifampicin
Isoniazid
Pyrazinamide
Ethambutol
Streptomycin
Moxifloxacin
Levofloxacin
Bedaquiline
Clofazimine
Linezolid
Cycloserine
Cycloserine
67
    Antibiotique Interprétation   Mutation
0       Amikacin        INCONNU    rrs 225
1       Amikacin        INCONNU    rrs 313
2       Amikacin        INCONNU   rrs 1029
3       Amikacin    Indeterminé  eis g-10c
4    Bedaquiline       Sensible           
..           ...            ...        ...
63  Streptomycin        INCONNU    rrs 313
64  Streptomycin        INCONNU   rrs 1029
65  Streptomycin       Sensible           
66  Streptomycin       Sensible           
67   Cycloserine       Sensible           

[68 rows x 3 columns]
Delamanid
Pretomanide
Pretomanide
68
    Antibiotique Interprétation   Mutation
0       Amikacin        INCONNU    rrs 225
1       Amikacin        INCONNU    rrs 313
2       Amikacin        INCONNU   rrs 1029
3       Amikacin    Indeterminé  eis g-10c
4    Bedaquiline       Sensible           
..           ...     

In [343]:
table_for_interpretation_to_order=table_for_interpretation_temp.drop_duplicates()
table_for_interpretation_to_order


table_for_interpretation_to_order['Antibiotique'] = pd.Categorical(table_for_interpretation_to_order['Antibiotique'], categories = order)
table_for_interpretation=table_for_interpretation_to_order.sort_values(by = 'Antibiotique')
table_for_interpretation

/tmp/ipykernel_237560/1620129468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_for_interpretation_to_order['Antibiotique'] = pd.Categorical(table_for_interpretation_to_order['Antibiotique'], categories = order)


,Antibiotique,Interprétation,Mutation
54,Rifampicin,INCONNU,rpoB S450S
53,Rifampicin,Résistant,rpoB S450F
55,Rifampicin,Sensible,
57,Rifampicin,Indeterminé,Rv2752c L11fs
58,Rifampicin,INCONNU,Rv2752c G9G
26,Isoniazid,Indeterminé,Rv2752c L11fs
25,Isoniazid,Résistant,katG S315T
21,Isoniazid,Sensible,
27,Isoniazid,INCONNU,Rv2752c G9G
51,Pyrazinamide,Résistant,pncA V131fs


In [344]:

def formating_for_print(x):
    #print(order)
    for cle in order:
        if cle in x.Antibiotique.unique() :
            print(cle)
            if "Résistant" in x.loc[x['Antibiotique']==cle].Interprétation.unique():
                x.drop(x[(x['Antibiotique']==cle) & (x['Interprétation']!="Résistant")].index, inplace=True )
            elif "INCONNU" in x.loc[x['Antibiotique']==cle].Interprétation.unique():
                x.drop(x[(x['Antibiotique']==cle) & (x['Interprétation']=="Sensible")].index, inplace=True )
            elif "Indeterminé" in x.loc[x['Antibiotique']==cle].Interprétation.unique(): 
                x.drop(x[(x['Antibiotique']==cle) & (x['Interprétation']=="Sensible")].index, inplace=True )
    print(x.Antibiotique.unique())

formating_for_print(table_for_interpretation)
#table_for_interpretation




#table_for_interpretation.loc[(table_for_interpretation['Antibiotique']=="Cycloserine") & (table_for_interpretation['Interprétation']=="INCONNU")].Mutation
#table1_step1_sansNa_sorted.loc[table1_step1_sansNa_sorted[table1_step1_sansNa_sorted.Antibiotique=="Streptomycin"].Interprétation=="RESISTANT"]

Rifampicin
Isoniazid
Pyrazinamide
Ethambutol
Streptomycin
Moxifloxacin
Levofloxacin
Bedaquiline
Clofazimine
Linezolid
Cycloserine
Delamanid
Pretomanide
Kanamycin
Amikacin
Capreomycin
Ethionamide
PAS
['Rifampicin', 'Isoniazid', 'Pyrazinamide', 'Ethambutol', 'Streptomycin', ..., 'Kanamycin', 'Amikacin', 'Capreomycin', 'Ethionamide', 'PAS']
Length: 18
Categories (18, object): ['Rifampicin', 'Isoniazid', 'Pyrazinamide', 'Ethambutol', ..., 'Amikacin', 'Capreomycin', 'Ethionamide', 'PAS']


In [345]:
table1_resistance_Mutaggregate=table_for_interpretation.groupby(['Antibiotique']).agg(lambda x: '; '.join(x))
table1_resistance_Mutaggregate


,Interprétation,Mutation
Antibiotique,,
Rifampicin,Résistant,rpoB S450F
Isoniazid,Résistant,katG S315T
Pyrazinamide,Résistant,pncA V131fs
Ethambutol,Résistant,embB M306V
Streptomycin,Résistant,rpsL K43R
Moxifloxacin,Résistant; Résistant,gyrA D94G; gyrA DS94GT
Levofloxacin,Résistant; Résistant,gyrA DS94GT; gyrA D94G
Bedaquiline,Sensible,
Clofazimine,Sensible,


In [346]:
#table1_resistance_Mutaggregate.columns
table1_to_print =table1_resistance_Mutaggregate.rename(columns={'Interprétation':'Résistance génotypique'   , 'Mutation' : 'Mutations' })
table1_to_print.to_csv(path_table1_toWrite, sep='\t', index=True)
table1_to_print.to_excel(path_table1_toWriteExcel)




NameError: name 'path_table1_toWrite' is not defined